In [ ]:
import pandas as pd
import numpy as np
#C:\Users\Acer\Documents\DataSprint\data\DataWaveDataSet.csv
file = r"C:\Users\Acer\Documents\DataSprint\data\DataWaveDataSet.csv"

df = pd.read_csv(file)

df.head()
df.info()
df.shape
df.describe()


df.isnull().sum()

In [15]:
df['user_id'].duplicated().sum()

df = df.drop_duplicates(subset=['user_id'])

df = df.dropna(subset=['user_id'])